In [43]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import string

# Read recipe inputs
ACCOUNTS_WITH_BUNDLER_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_BUNDLER_AND_DUNS")
ACCOUNTS_WITH_BUNDLER_AND_DUNS_df = ACCOUNTS_WITH_BUNDLER_AND_DUNS.get_dataframe()
df = ACCOUNTS_WITH_BUNDLER_AND_DUNS_df

/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:413: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 51868), raddr=('10.92.106.166', 10001)>
  return _handle_json_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)
/data/dataiku/dss_data/code-envs/python/env_clc/lib/python3.6/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 51870), raddr=('10.92.106.166', 10001)>
  if __name__ == '__main__':


In [60]:
df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,DUNS,IS_BUNDLER,DNB_GLOBAL_ULT_NUMBER,DNB_GLOBAL_ULT_NAME,DNB_DOMESTIC_ULT_NUMBER,DNB_DOMESTIC_ULTIMATE_NAME,DNB_HQ_NUMBER,DNB_HQ_NAME,DNB_DUNS_NUMBER,DNB_BUSINESS_NAME,DNB_LEVEL,DNB_CUSTOMER_NAME,EDW_STATE,CUSTOMER,CUST_CALC_SOURCE
0,9100009729900,RENZO EXCAVATING LLC,NaN,77302467,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,77302467,RENZO EXCAVATING LLC,DUNS,RENZO EXCAVATING LLC,None,RENZO EXCAVATING LLC,DNB
1,9100004871322,HAWKINS PEST CONTROL,NaN,879581114,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,879581114,HAWKINS PEST CONTROL INC,DUNS,HAWKINS PEST CONTROL INC,None,HAWKINS PEST CONTROL INC,DNB
2,9100006919713,HAWKINS PEST CONTROL INC,NaN,879581114,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,879581114,HAWKINS PEST CONTROL INC,DUNS,HAWKINS PEST CONTROL INC,None,HAWKINS PEST CONTROL INC,DNB
3,9100010883313,WATERMARK LIVERMORE LLC,NaN,606673127,True,802885111,FOUNTAINS AFFILIATED COMPANIES INC,802885111,FOUNTAINS AFFILIATED COMPANIES INC,802885111,FOUNTAINS AFFILIATED COMPANIES INC,606673127,WATERMARK RETIREMENT COMMUNITIES INC,DUNS Global,FOUNTAINS AFFILIATED COMPANIES INC,None,FOUNTAINS AFFILIATED COMPANIES INC,DNB
4,410008532954,FOUNTAINS LAKE POINTE WOODS SL,FOUNTAINS LAKE POINTE WOODS SL,606673127,False,802885111,FOUNTAINS AFFILIATED COMPANIES INC,802885111,FOUNTAINS AFFILIATED COMPANIES INC,802885111,FOUNTAINS AFFILIATED COMPANIES INC,606673127,WATERMARK RETIREMENT COMMUNITIES INC,DUNS Global,FOUNTAINS AFFILIATED COMPANIES INC,Set,FOUNTAINS LAKE POINTE WOODS SL,EDW


In [61]:
import warnings
warnings.filterwarnings(action='once')

df['DUNS'] = df['DUNS'].astype('Int64', errors='ignore')
df['DNB_DUNS_NUMBER'] = df['DNB_DUNS_NUMBER'].astype('Int64', errors='ignore')
df['DNB_BUSINESS_NAME'] = df['DNB_BUSINESS_NAME'].str.upper()
df["DNB_BUSINESS_NAME"] = df['DNB_BUSINESS_NAME'].str.replace('[^\w\s]','')

df['DNB_GLOBAL_ULT_NUMBER'] = df['DNB_GLOBAL_ULT_NUMBER'].astype('Int64', errors='ignore')
df['DNB_GLOBAL_ULT_NAME'] = df['DNB_GLOBAL_ULT_NAME'].str.upper()
df["DNB_GLOBAL_ULT_NAME"] = df['DNB_GLOBAL_ULT_NAME'].str.replace('[^\w\s]','')

df['DNB_DOMESTIC_ULT_NUMBER'] = df['DNB_DOMESTIC_ULT_NUMBER'].astype('Int64', errors='ignore')
df['DNB_DOMESTIC_ULTIMATE_NAME'] = df['DNB_DOMESTIC_ULTIMATE_NAME'].str.upper()
df["DNB_DOMESTIC_ULTIMATE_NAME"] = df['DNB_DOMESTIC_ULTIMATE_NAME'].str.replace('[^\w\s]','')

df['DNB_HQ_NUMBER'] = df['DNB_HQ_NUMBER'].astype('Int64', errors='ignore')
df['DNB_HQ_NAME'] = df['DNB_HQ_NAME'].str.upper()
df["DNB_HQ_NAME"] = df['DNB_HQ_NAME'].str.replace('[^\w\s]','')

df['DNB_LEVEL'] = 'None'
df['DNB_CUSTOMER_NAME'] = np.nan

df.loc[~df["DNB_GLOBAL_ULT_NAME"].isnull(),'DNB_LEVEL'] = "DUNS Global"
df.loc[~df["DNB_GLOBAL_ULT_NAME"].isnull(),'DNB_CUSTOMER_NAME'] = df.DNB_GLOBAL_ULT_NAME

df.loc[(df["DNB_GLOBAL_ULT_NAME"].isnull())&(~df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull()),'DNB_LEVEL'] = "DUNS Domestic"
df.loc[(df["DNB_GLOBAL_ULT_NAME"].isnull())&(~df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull()),'DNB_CUSTOMER_NAME'] = df.DNB_DOMESTIC_ULTIMATE_NAME

df.loc[(df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull())&(~df["DNB_HQ_NAME"].isnull()),'DNB_LEVEL'] = "DUNS HQ"
df.loc[(df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull())&(~df["DNB_HQ_NAME"].isnull()),'DNB_CUSTOMER_NAME'] = df.DNB_HQ_NAME

df.loc[(df["DNB_HQ_NAME"].isnull())&(~df["DNB_BUSINESS_NAME"].isnull()),'DNB_LEVEL'] = "DUNS"
df.loc[(df["DNB_HQ_NAME"].isnull())&(~df["DNB_BUSINESS_NAME"].isnull()),'DNB_CUSTOMER_NAME'] = df.DNB_BUSINESS_NAME

df['EDW_STATE'] = 'Unknown'
df.loc[df["EDW_CUSTOMER_NAME"].isnull(),'EDW_STATE'] = "None"
df.loc[~df["EDW_CUSTOMER_NAME"].isnull(),'EDW_STATE'] = "Set"

df['CUSTOMER'] = np.nan
df['CUST_CALC_SOURCE'] = 'Unknown'
df.loc[~df["EDW_CUSTOMER_NAME"].isnull(),'CUSTOMER'] = df["EDW_CUSTOMER_NAME"]
df.loc[~df["EDW_CUSTOMER_NAME"].isnull(),'CUST_CALC_SOURCE'] = "EDW"

df.loc[(df["CUSTOMER"].isnull())&(~df["DNB_CUSTOMER_NAME"].isnull()),'CUST_CALC_SOURCE'] = "DNB"
df.loc[df["CUSTOMER"].isnull(),'CUSTOMER'] = df["DNB_CUSTOMER_NAME"]

df.loc[df["CUSTOMER"].isnull(),'CUST_CALC_SOURCE'] = 'ACCOUNT'
df.loc[df["CUSTOMER"].isnull(),'CUSTOMER'] = df["CUSTOMER_ACCOUNT_NAME"]

# RULE SETs
# Rule 001
df.loc[(df['CUSTOMER']=='QUANTA SERVICES INC')|(df['CUSTOMER']=='QUANTA SERVICES'),"CUSTOMER"] = "QUANTA SERVICES INC"
df.loc[(df['CUSTOMER']=='QUANTA SERVICES INC')|(df['CUSTOMER']=='QUANTA SERVICES'),"CUST_CALC_SOURCE"] = "RULE 001"

# Rule 002
df.loc[(df['CUSTOMER']=='0113 WINDSTREAM COMM')|(df['CUSTOMER']=='0113 WINDSTREAM COMM (2)'),"CUSTOMER"] = "0113 WINDSTREAM COMM"
df.loc[(df['CUSTOMER']=='0113 WINDSTREAM COMM')|(df['CUSTOMER']=='0113 WINDSTREAM COMM (2)'),"CUST_CALC_SOURCE"] = "RULE 002"

df.CUST_CALC_SOURCE.value_counts()

<>:7: DeprecationWarning: invalid escape sequence \w


DNB         393821
EDW         241006
ACCOUNT     209282
RULE 001       306
RULE 002         2
Name: CUST_CALC_SOURCE, dtype: int64

In [59]:
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df = df

# Write recipe outputs
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS")
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS.write_with_schema(ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df)

/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:437: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 41080), raddr=('10.92.106.166', 10001)>
  return _handle_void_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)
/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:413: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 41082), raddr=('10.92.106.166', 10001)>
  return _handle_json_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)


844417 rows successfully written (mxK7tLybc0)


/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:437: ResourceWarning: unclosed <socket.socket fd=58, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 41084), raddr=('10.92.106.166', 10001)>
  return _handle_void_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)
/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:425: ResourceWarning: unclosed <socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 41086), raddr=('10.92.106.166', 10001)>
  return _handle_json_resp(backend_api_get_call(path, data, **kwargs), err_msg = err_msg)
